# Black-Scholes Equations

**Fall 2025 Quantitative Methods in Finance**

**The Erdős Institute**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


#Package import for normal distribution
from scipy.stats import norm

sns.set_style('darkgrid')

### Call Option

A **European call option** is a contract between two parties, a buyer and a seller, that gives the buyer the right, but not the obligation, to buy an underlying asset from the seller for a predetermined price $K$, called the **strike price**, at a future point in time.


### Put Option

A **European put option** is a contract between two parties, a buyer and a seller, that gives the buyer the right, but not the obligation, to sell an underlying asset to the seller for a predetermined price $K$, called the **strike price**, at a future point in time.


### Option Terminology
- **Premium**: The price paid by the option buyer to the option seller for obtaining the right. It represents the cost of the option.
- **Strike Price**: The predetermined price at which the underlying asset can be bought or sold.
- **Expiration Date**: The date when the option contract expires. After this date, the option is no longer valid.
- **Time to Expiration**: The amount of time in years to the contract expiration date.
- **In-the-Money (ITM)**: For a call option, when the market price is above the strike price. For a put option, when the market price is below the strike price.
- **Out-of-the-Money (OTM)**: For a call option, when the market price is below the strike price. For a put option, when the market price is above the strike price.
- **At-the-Money (ATM)**: When the market price is equal to the strike price.
- **Spot Price/Asset Price**: Price of the underlying asset.

### Black-Scholes Equation for Call and Put Options

Let $t > 0$ and assume the distribution of stock prices $S_t$ from time $0$ to time $t$ is the risk-free Geometric Brownian Motion model:

$$
S_t = S_0e^{\left(r - \frac{\sigma^2}{2}\right)t + \sigma\sqrt{t}\,\mathcal{N}(0,1)}.
$$

Where:
- $S_0$ is the stock price at time $0$;
- $\sigma$ is yearly standard deviation of log-returns;
- $r$ is the risk-free interest rate;
- $\mathcal{N}(0,1)$ is the standard normal distribution.

Let $\varphi$ and $\Phi$ denote the probability density function (PDF) and cumulative distribution function (CDF) of the standard normal distribution respectively:

$$
\varphi(x) = \frac{e^{-x^2/2}}{\sqrt{2\pi}} \quad \text{and} \quad \Phi(y) = \mathbb{P}(\mathcal{N}(0,1) \leq y) = \int_{-\infty}^y\varphi(x) \, dx.
$$

Let $K > 0$ be a strike price. Denote by $C_t = \max(S_t-K,0)$ and $P_t = \max(K-S_t,0)$ the distribution of respective values of a call and a put option at expiration time $t$. Let 

$$C_0 = e^{-rt}\mathbb{E}[\max(S_t-K,0)] \quad \text{and} \quad P_0 = e^{-rt}\mathbb{E}[\max(K-S_t,0)]$$

be the discounted to time $0$ expected values of $C_t$ and $P_t$ respectively. Then

$$
C_0 = S_0\Phi(d_1) - K e^{-rt}\Phi(d_2) \quad \text{and} \quad P_0 = -S_0\Phi(-d_1) + K e^{-rt}\Phi(-d_2),
$$

where

$$
d_1 = \frac{\ln\left(\frac{S_0}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)t}{\sigma \sqrt{t}} \quad \text{and} \quad
d_2 = d_1 - \sigma\sqrt{t}.
$$

The probability that $C_t>0$ and $P_t>0$ respectively is 

$$\mathbb{P}[C_t>0] = \Phi(d_2) \quad \text{and} \quad \mathbb{P}[P_t>0] = \Phi(-d_2).$$

**Remark**:

The above formulas assume that option prices can be determined using a **risk-neutral** valuation framework. This assumption can be justified under the following assumptions:

- A trader can continuously buy and sell fractional shares of the stock to construct a dynamic hedging portfolio that replicates the option payoff.

- There are no transaction costs or bid-ask spreads.

In [7]:
# Create functions for Black-Scholes option equations
import numpy as np
from scipy.stats import norm

def bs_call(S0, K, sigma, t, r):
    '''
    Black-Scholes Call Option formula
    
    Inputs:
    S0 (float): Stock price at time 0
    K (float): Strike Price
    sigma: Yearly volatility
    t: Time to expiration (years)
    r: Risk-free Interest rate
    
    
    Return:
    Black-Scholes value of call option (float)
    '''
    
    # Calculate d1 and d2
    d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*t) / (sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    
    # Calculate call option price
    call_price = S0*norm.cdf(d1) - K*np.exp(-r*t)*norm.cdf(d2)
    
    return call_price


def bs_put(S0, K, sigma, t, r):
    '''
    Black-Scholes Put Option formula
    
    Inputs:
    S0 (float): Stock price at time 0
    K (float): Strike Price
    sigma: Yearly volatility
    t: Time to expiration (years)
    r: Risk-free Interest rate
    
    
    Return:
    Black-Scholes value of put option (float)
    '''
    
    # Calculate d1 and d2
    d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*t) / (sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    
    # Calculate put option price
    put_price = -S0*norm.cdf(-d1) + K*np.exp(-r*t)*norm.cdf(-d2)
    
    return put_price

In [8]:
# Test functions
S0 = 100  # Current stock price
K = 105   # Strike price
sigma = 0.2  # Volatility
t = 0.25  # Time to expiration (3 months)
r = 0.05  # Risk-free rate

call_price = bs_call(S0, K, sigma, t, r)
put_price = bs_put(S0, K, sigma, t, r)

print(f"Call option price: ${call_price:.4f}")
print(f"Put option price: ${put_price:.4f}")
print(f"Stock price: ${S0}")
print(f"Strike price: ${K}")

Call option price: $2.4779
Put option price: $6.1736
Stock price: $100
Strike price: $105


## Call-Put Parity

Let $t > 0$ and assume the distribution of stock prices $S_t$ from time $0$ to time $t$ is the risk-free Geometric Brownian Motion model. Let $K > 0$ be a strike price of a call and put option with the same time to expiration $t$. Then 

$$C_0 - P_0 = S_0 - Ke^{-rt}.$$

In [9]:
# Check Call-Put parity
# According to call-put parity: C0 - P0 = S0 - e^(-rt)*K

left_side = call_price - put_price
right_side = S0 - np.exp(-r*t)*K

print(f"Call-Put Parity Check:")
print(f"C0 - P0 = {left_side:.6f}")
print(f"S0 - e^(-rt)*K = {right_side:.6f}")
print(f"Difference: {abs(left_side - right_side):.10f}")
print(f"Call-put parity holds: {abs(left_side - right_side) < 1e-10}")

Call-Put Parity Check:
C0 - P0 = -3.695669
S0 - e^(-rt)*K = -3.695669
Difference: 0.0000000000
Call-put parity holds: True


In [10]:
# Function for GBM stock paths
import numpy as np

def GBM_paths(S0, sigma, t, r, mu, n_sims, n_steps):
    """Simulates stock paths as geometric Brownian Motions
    Inputs:
    S0 (float): Underlying stock price at time 0
    sigma (float): Yearly volatility
    t (float): Time to expiration (years)
    r (float): Risk-free interest rate
    mu (float): Drift of log-returns
    n_sims (int): Number of simulated paths
    n_steps (int): Number of steps in each simulated path, each step interval has length t/n_steps
    
    Return (np.array): Array of stock paths
    """
    
    dt = t/n_steps
    noise = np.random.normal(loc=0, scale=1, size=(n_sims, n_steps))
    log_returns = (mu + r - sigma**2*0.5)*dt + sigma*np.sqrt(dt)*noise
    exponent = np.cumsum(log_returns, axis=1)
    paths = S0*np.exp(exponent)
    paths_with_start = np.insert(paths, 0, S0, axis=1)

    return paths_with_start

In [ ]:
# Test Monte-Carlo methods of call estimate against Black-Scholes
import matplotlib.pyplot as plt

# Monte Carlo simulation parameters
n_sims = 10000
n_steps = 252
mu = r  # Risk-neutral valuation

# Generate stock paths
paths = GBM_paths(S0, sigma, t, r, mu, n_sims, n_steps)

# Calculate option payoffs at expiration
final_prices = paths[:, -1]  # Final stock prices
call_payoffs = np.maximum(final_prices - K, 0)
put_payoffs = np.maximum(K - final_prices, 0)

# Calculate option prices (discounted expected payoffs)
mc_call_price = np.exp(-r*t) * np.mean(call_payoffs)
mc_put_price = np.exp(-r*t) * np.mean(put_payoffs)

# Compare with Black-Scholes
print(f"Monte Carlo vs Black-Scholes Comparison:")
print(f"Call Option:")
print(f"  Black-Scholes: ${call_price:.4f}")
print(f"  Monte Carlo:   ${mc_call_price:.4f}")
print(f"  Difference:    ${abs(call_price - mc_call_price):.4f}")
print()
print(f"Put Option:")
print(f"  Black-Scholes: ${put_price:.4f}")
print(f"  Monte Carlo:   ${mc_put_price:.4f}")
print(f"  Difference:    ${abs(put_price - mc_put_price):.4f}")

# Plot some sample paths
plt.figure(figsize=(12, 6))
time_steps = np.linspace(0, t, n_steps + 1)
for i in range(min(50, n_sims)):
    plt.plot(time_steps, paths[i, :], alpha=0.3, color='blue')
plt.axhline(y=K, color='red', linestyle='--', label=f'Strike Price = ${K}')
plt.xlabel('Time (years)')
plt.ylabel('Stock Price ($)')
plt.title(f'Sample Stock Price Paths (Showing {min(50, n_sims)} out of {n_sims})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()